<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8aaebc8b89d04179395307b0dff7f1c28cbc82b6bd759954ccfb24dee1852f3f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 09:26:23
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -41.07 K (-0.29%)
Current PnL: -17.15 L (-11.53%)
CY Booked + Current PnL: -5.71 L (-3.84%)
-------------------
Total profit:  2.00 L
Total loss:  -19.15 L
-------------------
Total Booked + Current PnL: 21.27 L (17.48%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.0%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.66 L (57.83%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.47,196450.0,31780.0,11551.0,0.04,19.30,5.88,26.31,89.0,2.75,1.40,28.82,X40N,NTT,AC
76,TTKPRESTIG,770.00,101.28,49.0,M-SC,3.31,86735.0,-14042.0,14138.0,-0.33,-13.93,16.30,0.09,245.0,-0.99,0.62,13.95,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.60,112609.0,12105.0,17004.0,0.41,12.04,15.10,28.96,79.0,0.71,0.81,50.10,MH,ATH,METALS
83,VOLTAS,1530.00,0.04,53.0,H-MC,2.79,209985.0,18243.0,19508.0,-1.41,9.51,9.29,19.69,99.0,0.94,1.50,16.96,XY25,NTT,AC
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.83,219320.0,8954.0,19827.0,-0.11,4.26,9.04,13.68,37.0,0.45,1.57,21.48,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1166.35,75.0,M-SC,5.98,172328.0,-8815.0,54283.0,3.37,-4.87,31.50,25.10,235.0,-0.16,1.23,38.10,XY24,NTT,HEALTHCARE
79,UNITDSPR,1644.00,-11.52,58.0,H-LC,7.63,232114.0,-2030.0,50647.0,1.60,-0.87,21.82,20.77,86.0,-0.04,1.66,5.99,X40N,NTT,BREWERIES
31,HINDZINC,730.22,22.12,61.0,M-LC,1.66,199382.0,-5694.0,118253.0,1.17,-2.78,59.31,54.89,52.0,-0.05,1.43,20.65,X5K,ATH,METALS
34,ICICIPRULI,790.00,-20.08,50.0,H-MC,2.19,137600.0,1812.0,40936.0,1.05,1.33,29.75,31.48,107.0,0.04,0.98,13.61,X40,ATH,INSURANCE
22,COLPAL,3767.14,0.72,51.0,H-MC,7.15,235130.0,-28235.0,141595.0,0.57,-10.72,60.22,43.04,84.0,-0.20,1.68,9.16,XY25,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.47,52.0,H-LC,1.45,239040.0,-7111.0,86317.0,-3.58,-2.89,36.11,32.18,16.0,-0.08,1.71,32.11,X200,ATH,IT
53,RAJESHEXPO,518.00,1798.99,53.0,L-SC,2.44,51797.0,-85380.0,85475.0,-3.49,-62.24,165.02,0.07,267.0,-1.00,0.37,28.82,OX40N,NTT,JEWELLERY
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT
27,HAPPSTMNDS,1488.71,-16.05,41.0,H-SC,10.31,90648.0,-37517.0,147548.0,-2.49,-29.27,162.77,85.85,132.0,-0.25,0.65,4.89,AR,ATH,IT
30,HCLTECH,1943.91,-1.51,40.0,H-LC,9.86,224793.0,-17131.0,80408.0,-2.43,-7.08,35.77,26.15,8.0,-0.21,1.61,6.23,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.23,57.0,H-SC,16.77,201922.0,1968.0,95267.0,-0.15,0.98,47.18,48.63,115.0,0.02,1.44,12.65,XR,NTT,MISC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248383.0,652.0,74416.0,0.09,0.26,29.96,30.30,92.0,0.01,1.78,13.59,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.32,122877.0,-19602.0,39124.0,-0.61,-13.76,31.84,13.70,163.0,-0.50,0.88,45.03,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,101.28,49.0,M-SC,3.31,86735.0,-14042.0,14138.0,-0.33,-13.93,16.30,0.09,245.0,-0.99,0.62,13.95,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,59.0,H-SC,0.83,225198.0,-44469.0,80801.0,-0.65,-16.49,35.88,13.47,138.0,-0.55,1.61,13.90,XY24,NTT,PAINTS
18,CERA,9475.0,-21.00,40.0,H-SC,2.17,143830.0,-32073.0,74101.0,-0.17,-18.23,51.52,23.89,149.0,-0.43,1.03,24.22,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-20.85,48.0,H-MC,7.26,106989.0,-24846.0,67071.0,-1.60,-18.85,62.69,32.03,98.0,-0.37,0.77,19.84,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,47.55,67.0,M-MC,8.16,233004.0,8042.0,159608.0,-0.65,3.57,68.50,74.53,192.0,0.05,1.67,36.98,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.47,196450.0,31780.0,11551.0,0.04,19.30,5.88,26.31,89.0,2.75,1.40,28.82,X40N,NTT,AC
17,CAMS,5211.76,1.26,68.0,H-SC,2.98,115052.0,13048.0,30880.0,0.20,12.79,26.84,43.06,122.0,0.42,0.82,32.10,X40N,ATH,MISC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248383.0,652.0,74416.0,0.09,0.26,29.96,30.30,92.0,0.01,1.78,13.59,X40,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.04,53.0,H-MC,2.79,209985.0,18243.0,19508.0,-1.41,9.51,9.29,19.69,99.0,0.94,1.50,16.96,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.60,112609.0,12105.0,17004.0,0.41,12.04,15.10,28.96,79.0,0.71,0.81,50.10,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,0.84,100522.0,9024.0,69732.0,0.19,9.86,69.37,86.07,223.0,0.13,0.72,45.98,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158646.0,7701.0,101756.0,-0.41,5.10,64.14,72.51,53.0,0.08,1.13,11.43,XR,NTT,IT
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.62,186313.0,6505.0,105900.0,0.56,3.62,56.84,62.52,88.0,0.06,1.33,38.94,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,33.0,H-SC,11.35,130009.0,6673.0,127084.0,-0.67,5.41,97.75,108.45,119.0,0.05,0.93,28.97,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HCLTECH,1943.91,-1.51,40.0,H-LC,9.86,224793.0,-17131.0,80408.0,-2.43,-7.08,35.77,26.15,8.0,-0.21,1.61,6.23,X40,ATH,IT
82,VBL,671.64,-15.83,43.0,H-LC,6.77,302986.0,-12856.0,125527.0,0.09,-4.07,41.43,35.67,5.0,-0.10,2.17,9.22,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.19,47.0,H-LC,7.75,219199.0,-32569.0,85773.0,0.37,-12.94,39.13,21.13,27.0,-0.38,1.57,17.91,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200343.0,205.0,20675.0,-0.23,0.10,10.32,10.44,4.0,0.01,1.43,5.67,X40,NTT,FMCG
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-3.47,52.0,H-LC,1.45,239040.0,-7111.0,86317.0,-3.58,-2.89,36.11,32.18,16.0,-0.08,1.71,32.11,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.25,200343.0,205.0,20675.0,-0.23,0.10,10.32,10.44,4.0,0.01,1.43,5.67,X40,NTT,FMCG
64,SIEMENS,4671.5,2.53,60.0,H-LC,3.56,164670.0,-21425.0,68898.0,-0.98,-11.51,41.84,25.51,15.0,-0.31,1.18,21.35,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-8.86,54.0,H-LC,4.20,280613.0,15187.0,41615.0,0.39,5.72,14.83,21.40,11.0,0.36,2.01,12.74,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,55.0,H-LC,4.83,219320.0,8954.0,19827.0,-0.11,4.26,9.04,13.68,37.0,0.45,1.57,21.48,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200343.0,205.0,20675.0,-0.23,0.10,10.32,10.44,4.0,0.01,1.43,5.67,X40,NTT,FMCG
79,UNITDSPR,1644.00,-11.52,58.0,H-LC,7.63,232114.0,-2030.0,50647.0,1.60,-0.87,21.82,20.77,86.0,-0.04,1.66,5.99,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.51,40.0,H-LC,9.86,224793.0,-17131.0,80408.0,-2.43,-7.08,35.77,26.15,8.0,-0.21,1.61,6.23,X40,ATH,IT
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.53,60.0,H-LC,3.56,164670.0,-21425.0,68898.0,-0.98,-11.51,41.84,25.51,15.0,-0.31,1.18,21.35,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200343.0,205.0,20675.0,-0.23,0.10,10.32,10.44,4.0,0.01,1.43,5.67,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,54.0,H-LC,5.59,215446.0,10946.0,30464.0,-0.20,5.35,14.14,20.25,10.0,0.36,1.54,14.55,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.19,47.0,H-LC,7.75,219199.0,-32569.0,85773.0,0.37,-12.94,39.13,21.13,27.0,-0.38,1.57,17.91,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.83,219320.0,8954.0,19827.0,-0.11,4.26,9.04,13.68,37.0,0.45,1.57,21.48,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319670.0,6782.0,164918.0,-2.56,2.17,51.59,54.87,3.0,0.04,2.29,8.96,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.25,200343.0,205.0,20675.0,-0.23,0.10,10.32,10.44,4.0,0.01,1.43,5.67,X40,NTT,FMCG
82,VBL,671.64,-15.83,43.0,H-LC,6.77,302986.0,-12856.0,125527.0,0.09,-4.07,41.43,35.67,5.0,-0.10,2.17,9.22,X40N,ATH,FMCG
1,ABB,7934.00,-37.82,65.0,H-LC,9.41,259896.0,-1723.0,120930.0,-0.41,-0.66,46.53,45.57,7.0,-0.01,1.86,10.59,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7022.22,65.0,L-SC,5.15,78123.0,-15687.0,92302.0,-0.24,-16.72,118.15,81.67,269.0,-0.17,0.56,51.29,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.42,84346.0,-29203.0,69257.0,-0.88,-25.72,82.11,35.27,268.0,-0.42,0.60,106.44,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-172.73,63.0,M-SC,3.88,84018.0,-18192.0,111584.0,-2.22,-17.80,132.81,91.37,208.0,-0.16,0.60,36.39,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.00,48.0,H-SC,12.28,89277.0,-11686.0,106222.0,-0.09,-11.57,118.98,93.63,148.0,-0.11,0.64,32.98,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,50.0,H-SC,12.61,94125.0,-3855.0,25461.0,-0.51,-3.93,27.05,22.05,152.0,-0.15,0.67,36.82,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1166.35,75.0,M-SC,5.98,172328.0,-8815.0,54283.0,3.37,-4.87,31.50,25.10,235.0,-0.16,1.23,38.10,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-14.17,78.0,H-SC,13.54,230173.0,-40588.0,370532.0,-0.20,-14.99,160.98,121.86,133.0,-0.11,1.65,80.06,AR,ATH,IT
7,ATULAUTO,844.00,3797.30,59.0,M-SC,4.93,121552.0,-25816.0,77635.0,0.18,-17.52,63.87,35.16,236.0,-0.33,0.87,24.15,XY24,NTT,AUTO
60,SAMMAANCAP,326.00,-172.73,63.0,M-SC,3.88,84018.0,-18192.0,111584.0,-2.22,-17.80,132.81,91.37,208.0,-0.16,0.60,36.39,XY25,NTT,FINANCE
26,GREENPANEL,537.00,231.40,60.0,M-SC,1.14,147937.0,-35141.0,114119.0,-0.80,-19.19,77.14,43.14,230.0,-0.31,1.06,39.16,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,44.22
2,50,75.22


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.85
LC    31.28
MC    23.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.79
X40      14.97
X40N     11.68
XR       11.48
AR        8.61
XY25      8.40
OX40N     7.97
X200      1.71
MH        1.68
X5K       1.43
SR        1.25
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.99
H-LC    24.68
H-MC    20.89
M-SC    13.33
M-LC     5.56
M-MC     2.65
L-SC     1.53
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.24,-13.64,75.60
FMCG,12.35,-3.98,41.18
FINANCE,10.22,-14.65,64.68
MISC,7.24,-13.44,76.51
BANKS,6.23,-14.55,74.81
PAINTS,5.71,-15.65,32.89
ELECTRICAL,5.62,-4.93,41.74
INSURANCE,3.72,-6.01,45.64
AC,3.60,4.12,11.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3073317.0,22
XR,1301845.0,14
AR,1128669.0,9
X40,755497.0,10
X40N,570267.0,9
OX40N,533346.0,10
XY25,372264.0,6
SR,246433.0,2
X5K,118253.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3596575.0,29
M-SC,1379426.0,17
H-MC,1183844.0,15
H-LC,1079334.0,14
M-LC,373573.0,4
M-MC,307888.0,2
L-SC,247034.0,3
L-MC,60357.0,1
L-LC,38135.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1207309.0      6
M-SC       XY24       794409.0      7
H-SC       AR         776774.0      5
           XR         758575.0      7
H-MC       XY24       549728.0      4
H-LC       X40        474690.0      5
M-MC       XY24       307888.0      2
H-SC       X40N       290198.0      4
           OX40N      254332.0      4
           SR         246433.0      2
H-MC       X40        209607.0      4
H-LC       X40N       206638.0      3
           AR         189828.0      2
M-SC       AR         162067.0      2
L-SC       XR         161559.0      2
H-MC       XY25       161103.0      2
M-LC       XY24       153564.0      2
M-SC       OX40N      126468.0      4
M-LC       X5K        118253.0      1
M-SC       XR         113698.0      2
           XY25       111584.0      1
H-MC       XR         105900.0      1
M-LC       XR         101756.0      1
H-LC       X200        86317.0      1
L-SC       OX40N       85475.0      1
H-MC       X40N        73431.0      2
M-SC       X40         71200.0      1
H-MC       OX40N       67071.0      1
H-SC       MH          62954.0      1
H-LC       XY25        61442.0      2
           XY24        60419.0      1
L-MC       XR          60357.0      1
L-LC       XY25        38135.0      1
H-MC       MH          17004.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
